In [1]:
from src.methods import *
import json
import pandas as pd
import time
import numpy as np
from src.text import TextProcessor
import os
from src.utils import idx_to_docid, recall_at_10, get_class

In [2]:
import re

In [3]:
corpus = json.load(open('../dis-project-1-document-retrieval/corpus.json/corpus.json', 'r'))
dev_df = pd.read_csv('../dis-project-1-document-retrieval/dev.csv')


In [5]:
import pickle
korean_text = pickle.load(open('korean_text.pkl', 'rb'))

In [6]:
lang = 'ko'
my_corpus = [x for x in corpus if x['lang'] == lang]

In [8]:
for i in range(len(my_corpus)):
    my_corpus[i]['text'] = ' '.join(korean_text[i])

In [9]:
my_corpus[0]['text']

'강감찬 姜邯贊 또는 姜邯瓚 948년 음력 11월 19일 1031년 음력 8월 20일 고려 의 문관 이다 본관 은 금주 이다 983년 성종 과거 에 장원급제 하여 예 부시 랑 한림 학사 를 거쳐 문 하평 장사 가 되었다 요나라 의 3 차 침입 을 격퇴 하였고 천수현 개국 남 天水縣開國男 봉 해졌다 1030년 정종 21 문하시중 에 이르렀다 현종 의 묘정 廟廷 배향 되고 문종 文宗 수 태사 守太師 중 서 령 中書令 추증 되었다 시호 는 인헌 仁憲 이다 생애 생애 강감찬 은 948년 12월 22일 음력 11월 19일 삼 한 벽 상 공신 三韓壁上功臣 강 궁진 의 아들 로 금주 지금 의 서울특별시 관악구 낙성대 근처 에서 태어났고 어릴 적 이름 은 은천 殷川 이지만 역사 적 으로는 강감찬 으로 남았다 5 대조 인 강 여 청 姜餘淸 시흥군 始興郡 으로 이주하 여 금주 衿州 호족 이 되었다 983년 성종 과거 에 장원 으로 급제 하여 예 부시 랑 이 되었다 거란 의 1 차 침공 993년 요나라 거란 성종 은 송나라 와 고려 사이 의 친선 관계 를 차단 하였다 또한 고구려 의 영토 를 자신 들 의 영토 라고 주장 하여 소손녕 蕭遜寧 보내 고려 를 침공 하였다 서희 徐熙 고구려 를 계승 하였기 때문 에 나라이름 을 고려 라고 하였고 그렇기 때문 에 요 의 땅 도 고려 땅 이라고 주장 압록강 동쪽 의 여진 을 내쫓고 땅 을 돌려 달라 고 하였다 그러자 말 에 설득 된 소손녕 은 물러 갔고 고려 는 서희 의 담판 결과 로써 강동 6 주 를 획득 하였다 거란 의 2 차 침공 1010년 현종 1년 요나라 성종 이 강조 의 정변 을 표면 상의 구실 로 내세워 40만 군 으로 서경 西京 침공 하였다 현종은 강조 를 행 영 도통 사 行營都統使 삼아 30만 군 을 거 느리고 통주 通州 나가 막게 했으나 크게 패배했다 강감찬 은 요나라 에 항복 하자는 다른 중신 들 에 반대 하며 전략 상 일시 후퇴 할 것 을 주장 하여 나주 로 피난 하여 사직 을 보호 하였다 하공진 河拱辰 적 을 설득 하

In [10]:
my_text = [x['text'] for x in my_corpus]

In [11]:
my_text[0]

'강감찬 姜邯贊 또는 姜邯瓚 948년 음력 11월 19일 1031년 음력 8월 20일 고려 의 문관 이다 본관 은 금주 이다 983년 성종 과거 에 장원급제 하여 예 부시 랑 한림 학사 를 거쳐 문 하평 장사 가 되었다 요나라 의 3 차 침입 을 격퇴 하였고 천수현 개국 남 天水縣開國男 봉 해졌다 1030년 정종 21 문하시중 에 이르렀다 현종 의 묘정 廟廷 배향 되고 문종 文宗 수 태사 守太師 중 서 령 中書令 추증 되었다 시호 는 인헌 仁憲 이다 생애 생애 강감찬 은 948년 12월 22일 음력 11월 19일 삼 한 벽 상 공신 三韓壁上功臣 강 궁진 의 아들 로 금주 지금 의 서울특별시 관악구 낙성대 근처 에서 태어났고 어릴 적 이름 은 은천 殷川 이지만 역사 적 으로는 강감찬 으로 남았다 5 대조 인 강 여 청 姜餘淸 시흥군 始興郡 으로 이주하 여 금주 衿州 호족 이 되었다 983년 성종 과거 에 장원 으로 급제 하여 예 부시 랑 이 되었다 거란 의 1 차 침공 993년 요나라 거란 성종 은 송나라 와 고려 사이 의 친선 관계 를 차단 하였다 또한 고구려 의 영토 를 자신 들 의 영토 라고 주장 하여 소손녕 蕭遜寧 보내 고려 를 침공 하였다 서희 徐熙 고구려 를 계승 하였기 때문 에 나라이름 을 고려 라고 하였고 그렇기 때문 에 요 의 땅 도 고려 땅 이라고 주장 압록강 동쪽 의 여진 을 내쫓고 땅 을 돌려 달라 고 하였다 그러자 말 에 설득 된 소손녕 은 물러 갔고 고려 는 서희 의 담판 결과 로써 강동 6 주 를 획득 하였다 거란 의 2 차 침공 1010년 현종 1년 요나라 성종 이 강조 의 정변 을 표면 상의 구실 로 내세워 40만 군 으로 서경 西京 침공 하였다 현종은 강조 를 행 영 도통 사 行營都統使 삼아 30만 군 을 거 느리고 통주 通州 나가 막게 했으나 크게 패배했다 강감찬 은 요나라 에 항복 하자는 다른 중신 들 에 반대 하며 전략 상 일시 후퇴 할 것 을 주장 하여 나주 로 피난 하여 사직 을 보호 하였다 하공진 河拱辰 적 을 설득 하

In [16]:
my_text[0]

'강감찬 姜邯贊 또는 姜邯瓚 948년 음력 11월 19일 1031년 음력 8월 20일 고려 의 문관 이다 본관 은 금주 이다 983년 성종 과거 에 장원급제 하여 예 부시 랑 한림 학사 를 거쳐 문 하평 장사 가 되었다 요나라 의 3 차 침입 을 격퇴 하였고 천수현 개국 남 天水縣開國男 봉 해졌다 1030년 정종 21 문하시중 에 이르렀다 현종 의 묘정 廟廷 배향 되고 문종 文宗 수 태사 守太師 중 서 령 中書令 추증 되었다 시호 는 인헌 仁憲 이다 생애 생애 강감찬 은 948년 12월 22일 음력 11월 19일 삼 한 벽 상 공신 三韓壁上功臣 강 궁진 의 아들 로 금주 지금 의 서울특별시 관악구 낙성대 근처 에서 태어났고 어릴 적 이름 은 은천 殷川 이지만 역사 적 으로는 강감찬 으로 남았다 5 대조 인 강 여 청 姜餘淸 시흥군 始興郡 으로 이주하 여 금주 衿州 호족 이 되었다 983년 성종 과거 에 장원 으로 급제 하여 예 부시 랑 이 되었다 거란 의 1 차 침공 993년 요나라 거란 성종 은 송나라 와 고려 사이 의 친선 관계 를 차단 하였다 또한 고구려 의 영토 를 자신 들 의 영토 라고 주장 하여 소손녕 蕭遜寧 보내 고려 를 침공 하였다 서희 徐熙 고구려 를 계승 하였기 때문 에 나라이름 을 고려 라고 하였고 그렇기 때문 에 요 의 땅 도 고려 땅 이라고 주장 압록강 동쪽 의 여진 을 내쫓고 땅 을 돌려 달라 고 하였다 그러자 말 에 설득 된 소손녕 은 물러 갔고 고려 는 서희 의 담판 결과 로써 강동 6 주 를 획득 하였다 거란 의 2 차 침공 1010년 현종 1년 요나라 성종 이 강조 의 정변 을 표면 상의 구실 로 내세워 40만 군 으로 서경 西京 침공 하였다 현종은 강조 를 행 영 도통 사 行營都統使 삼아 30만 군 을 거 느리고 통주 通州 나가 막게 했으나 크게 패배했다 강감찬 은 요나라 에 항복 하자는 다른 중신 들 에 반대 하며 전략 상 일시 후퇴 할 것 을 주장 하여 나주 로 피난 하여 사직 을 보호 하였다 하공진 河拱辰 적 을 설득 하

In [17]:
P = TextProcessor(lang, stopwords_version='v2')

In [18]:
corpus_tokens, corpus_map = P.preprocess_corpus(my_text)

100%|██████████| 7893/7893 [00:13<00:00, 594.94it/s] 


In [19]:
my_dev_df = dev_df[dev_df['lang'] == lang]
my_dev_set_queries = my_dev_df['query'].tolist()
my_dev_set_positive_docs = my_dev_df['positive_docs'].tolist()
query_tokens, query_map = P.preprocess_queries(my_dev_set_queries)

In [20]:
for method in ['BM25_V2']:
    # print('Starting', method)
    M = get_class(f'src.methods.{method}')
    # tf_idf = M(my_text, k1=1.5, b=0.75, d=1, save_path=f'dump/{lang}') # for all 
    tf_idf = M(my_text, k1=1.5, b=0.75, d=1, save_path=f'dump/{lang}') # for TF_LDP
    # print('Starting indexing')
    start = time.perf_counter()
    tf_idf.calculate_scores(corpus_tokens, corpus_map)
    end = time.perf_counter()
    # print(f'Indexing took {end - start}s.')
    # print(f'Saving Index and Vocab to dump/{lang}')
    # start = time.perf_counter()
    # tf_idf.save()
    # end = time.perf_counter()
    # print(f'Saving index took {end - start}s.')
    results, scores = tf_idf.parallel_search(query_tokens, query_map, topk=10, num_threads=1)
    results = idx_to_docid(results, my_corpus)
    print(method)
    recall_at_10(my_dev_set_positive_docs, results)

BM25_V2
0.35


In [30]:
from konlpy.tag import Okt
tokenizer = Okt()

In [37]:
import multiprocessing

In [38]:
okt = None


def multiprocessing_initializer():
    global okt
    okt = Okt()

In [39]:
def spacing_okt(wrongSentence):
    tagged = okt.pos(wrongSentence)
    corrected = ""
    for i in tagged:
        if i[1] in ('Josa', 'PreEomi', 'Eomi', 'Suffix', 'Punctuation'):
            corrected += i[0]
        else:
            corrected += " "+i[0]
    if corrected[0] == " ":
        corrected = corrected[1:]
    return corrected

In [40]:
from konlpy.corpus import kolaw
lines = set(map(lambda _: _.strip(), kolaw.open('constitution.txt').readlines()))
if '' in lines:
    lines.remove('')

In [42]:
from multiprocessing import Pool

In [43]:
with Pool(4, initializer=multiprocessing_initializer) as pool:
    print(pool.map(spacing_okt, lines))

Traceback (most recent call last):
  File "<string>", line 1, in <module>
Traceback (most recent call last):
  File "/opt/homebrew/Cellar/python@3.12/3.12.7_1/Frameworks/Python.framework/Versions/3.12/lib/python3.12/multiprocessing/spawn.py", line 122, in spawn_main
  File "<string>", line 1, in <module>
  File "/opt/homebrew/Cellar/python@3.12/3.12.7_1/Frameworks/Python.framework/Versions/3.12/lib/python3.12/multiprocessing/spawn.py", line 122, in spawn_main
Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/opt/homebrew/Cellar/python@3.12/3.12.7_1/Frameworks/Python.framework/Versions/3.12/lib/python3.12/multiprocessing/spawn.py", line 122, in spawn_main
    exitcode = _main(fd, parent_sentinel)
    exitcode = _main(fd, parent_sentinel)
    exitcode = _main(fd, parent_sentinel)
                                 ^ ^ ^ ^^^^^ ^^^^^^^ ^^ ^ ^ ^ ^ ^ ^ ^^^^^^^^^^^^^^^^^^
^^  File "/opt/homebrew/Cellar/python@3.12/3.12.7_1/Frameworks/Python.framework/Versions/3.

KeyboardInterrupt: 

In [49]:
okt.morphs('안녕하세요. 저는 박종수입니다.')

['안녕하세요', '.', '저', '는', '박종수', '입니다', '.']

In [47]:
okt = Okt()